In [ ]:
import time
import folium
import geopandas as gpd #A more flexible package to work with geospatial data in python 
from itertools import product
import networkx as nx # networkx package 
import numpy as np
from osgeo import ogr #GDAL package 
import osmnx as ox 
import pandas as pd #Base package for data analysis and manipulation 
from pyproj import CRS #package for your projection management 
import random
import requests
import shapely.geometry #python package for basic spatial operation 

### INTRODUCTION

The sections of this notebook are in order. 
The OSM settings should always be run.
Some of the operations in this notebook require data that can be downloaded from the data folder in this repository.

Downloading the AHN4 datasets can take quite some time, so make sure you have everything set up right before you do that. This also counts for downloading the networks.

The section with OSMOSIS can be ignored for now, but might be necessary for later if the downloading of network via OSMnx fails for some reason.

Adding grid value section is a work in progress.

### OSM settings

In [ ]:
#adds surface as extra tag for the edges (THIS IS A REQUIRED STEP BEFORE YOU DOWNLOAD THE NETWORK)
ox.settings.useful_tags_way.append('surface')

### RANDOM NODE GENERATOR

In [ ]:
def rand_sourcegoal_generator(G, limit):
    """
    function to generate a number of source and goal nodes in a given graph.
    
    Parameters:
    G (networkx.MultiDiGraph) - input graph
    limit (int) - total number of source and goal pairs to generate

    returns:
    sources (list) - source nodes
    goals (list) - goal nodes
    """
    nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)
    bbox = nodes.unary_union.envelope
    #min_lon = bbox.bounds[0]
    #min_lat = bbox.bounds[1]
    #max_lon = bbox.bounds[2]
    #max_lat = bbox.bounds[3]
    sources = []
    goals = []
    for n in range(limit):
        X1 = random.uniform(bbox.bounds[0], bbox.bounds[2])
        Y1 = random.uniform(bbox.bounds[1], bbox.bounds[3])
        X2 = random.uniform(bbox.bounds[0], bbox.bounds[2])
        Y2 = random.uniform(bbox.bounds[1], bbox.bounds[3])
        sources.append(ox.distance.nearest_nodes(G, X1, Y1))
        goals.append(ox.distance.nearest_nodes(G, X2, Y2))

    return sources, goals

### NETWERK EXTENT 

In [ ]:
#set the city names
g_namen = ['Amsterdam', 'Rotterdam', "Den_Haag", "Utrecht"]

#download city geometries from file
project_gemeenten = gpd.read_file(r"data\boundaries\UrbanRunner_Areas.geojson")


#set additional geometry column of extent
project_gemeenten["extent"] = project_gemeenten.envelope.buffer(2000).to_crs("EPSG:4326")

#set index to gemeentenaam column
project_gemeenten.set_index('gemeentenaam', inplace=True)

#project to WGS84
project_gemeenten.to_crs("EPSG:4326", inplace=True)
project_gemeenten['extent'].to_crs("EPSG:4326")


##### plot in folium

In [ ]:
map_centroid = project_gemeenten.unary_union.envelope.centroid.coords[0]

#create folium map
m = folium.Map(location = (map_centroid[1], map_centroid[0]))
#plot the convexhulls of each network region in the folium map
for _, r in project_gemeenten.iterrows():
    #unbuffered convex hull
    extent = gpd.GeoSeries(r["extent"])
    extent_j = extent.to_json()
    extent_j = folium.GeoJson(data=extent_j,
                           style_function=lambda x: {'fillColor': 'blue'})
    folium.Popup(r.name).add_to(extent_j)
    extent_j.add_to(m)

    #orginal geometry
    geom = gpd.GeoSeries(r['geometry'])
    geom_j = geom.to_json()
    geom_j = folium.GeoJson(data=geom_j,
                           style_function=lambda x: {'fillColor': 'green'})
    folium.Popup(r.name).add_to(geom_j)
    geom_j.add_to(m)

m

### DOWNLOADING AHN

In [ ]:
ahn_datavlakken = gpd.read_file(r"data/kaartbladen_AHN4.gpkg")
ahn_datavlakken.to_crs("EPSG:4326", inplace=True)
for g in g_namen:
    globals()[f"{g}_ahnvlakken"]=ahn_datavlakken.clip(project_gemeenten.loc[g].extent.envelope)

In [ ]:
for gemeente in g_namen:
    globals()[f"{gemeente}_ahnvlakken"]=ahn_datavlakken.clip(project_gemeenten.loc[gemeente].extent.envelope)
    for i in range(globals()[f"{gemeente}_ahnvlakken"].Name_1.count()):    
        name = globals()[f"{gemeente}_ahnvlakken"].iat[i,2]

        response = requests.get(globals()[f"{gemeente}_ahnvlakken"].iat[i,3])
        print('done with downloading number ' + i + ' of gemeente ' + gemeente)
        open('C:\\Users\\danny\\Documents\\persoonlijk\\GIMA\\modules\\module 6\\data\\AHN4_05M_DTM\\'+gemeente + '\\'+ name + '.zip', "wb").write(response.content)

In [ ]:
for g in g_namen:
    globals()[f"{g}_DTM_list"] = []
    for i in range(globals()[f"{g}_ahnvlakken"].Name_1.count()):    
        name = globals()[f"{g}_ahnvlakken"].iat[i,2]
        globals()[f"{g}_DTM_list"].append('C:\\Users\\danny\\Documents\\persoonlijk\\GIMA\\modules\\module 6\\data\\AHN4_05M_DTM\\'+ g + '\\M_'+ name +".tif")

In [ ]:
Utrecht_network = ox.elevation.add_node_elevations_raster(Utrecht_network, raster_list, band=1, cpus=2)

### GETTING NETWORK

In [ ]:
# simplify false
for g in g_namen:
    print('starting with', g)
    globals()[f"{g}_network_walk"] = ox.graph_from_polygon(project_gemeenten.loc[g].extent.envelope, network_type="walk", retain_all=True ,simplify=False)
    print('done with walking graph of', g)
    globals()[f"{g}_network_bike"] = ox.graph_from_polygon(project_gemeenten.loc[g].extent.envelope, network_type="bike", retain_all=True, simplify=False)
    print('done with biking graph of', g)
    globals()[f"{g}_network_both"] = nx.compose(globals()[f"{g}_network_walk"], globals()[f"{g}_network_bike"])

In [ ]:
for gemeente in g_namen:
    globals()[f"{gemeente}_network"] = ox.graph_from_polygon(project_gemeenten.loc[gemeente].convexhull, network_type="walk")
    globals()[f"{gemeente}_nodes"] = ox.utils_graph.graph_to_gdfs(globals()[f"{gemeente}_network"],nodes=True, edges=False, node_geometry=True)
    globals()[f"{gemeente}_edges"] = ox.utils_graph.graph_to_gdfs(globals()[f"{gemeente}_network"], nodes=False, edges=True, node_geometry=False)

### ROUTES

In [ ]:
def get_random_XY_in_polygon(poly):
    minx, miny, maxx, maxy = poly.bounds
    while True:
        p = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            return p.coords[0][0], p.coords[0][1]

In [ ]:
for g in g_namen:
    X_list = []
    Y_list = []    
    for i in range(100):
        x, y = get_random_XY_in_polygon(project_gemeenten.loc[g].geometry)
        X_list.append(x)
        Y_list.append(y)
    random_nodes = ox.nearest_nodes(NETWORK, X_list, Y_list, return_dist=True)

In [ ]:
#getting nodes from the random XY sets
#UNSIMPLIFIED
random_nodes = ox.nearest_nodes(u_network_both, X_list, Y_list, return_dist=True)
source_nodes = random_nodes[0][:50]
goal_nodes = random_nodes[0][50:100]
#SIMPLIFIED
random_nodes_simplified = ox.nearest_nodes(u_network_both_simplified, X_list, Y_list, return_dist=True)
source_nodes_simplified = random_nodes_simplified[0][:50]
goal_nodes_simplified = random_nodes_simplified[0][50:100]

In [ ]:
# calculating shortest route of the source and goal nodes
#UNSIMPLIFIED
start_time = time.time()
random_shortest_routes = ox.distance.shortest_path(u_network_both, source_nodes, goal_nodes, weight='length', cpus=1)
print("gathering random routes for unsimplified network takes %s seconds" % (time.time() - start_time))
randomroute_time = (time.time() - start_time)
#SIMPLIFIED
start_time = time.time()
random_shortest_routes_simplified = ox.distance.shortest_path(u_network_both_simplified, source_nodes_simplified, goal_nodes_simplified, weight='length', cpus=1)
print("gathering random routes for simplified network takes %s seconds" % (time.time() - start_time))
randomroute_time_simplified = (time.time() - start_time)

In [ ]:
ox.folium.plot_route_folium(u_network_both, random_shortest_routes[2], route_map=None, popup_attribute=None, tiles='cartodbpositron', zoom=1, fit_bounds=True)

In [ ]:
ox.folium.plot_route_folium(u_network_both_simplified, random_shortest_routes_simplified[2], route_map=None, popup_attribute=None, tiles='cartodbpositron', zoom=1, fit_bounds=True)

### USING OSMOSIS

### NODE ELEVATION

In [ ]:
for g in g_namen:
    globals()[f"{g}_network_both"] = ox.project_graph(globals()[f"{g}_network_both"], "epsg:28992")
    globals()[f"{g}_network_both"] = ox.elevation.add_node_elevations_raster(globals()[f"{g}_network_both"], globals()[f"{g}_DTM_list"], band=1, cpus=2)

### SAVING NETWORKS TO GEOPACKAGE

In [ ]:
for g in ['Den_Haag','Rotterdam','Utrecht']:
    ox.save_graph_geopackage(globals()[f"{g}_network_both"], filepath='data\\graphs\\' + g + '_Graph.gpkg')

### adding grid values to edges

In [ ]:
def make_grid_update(polygon, edge_size):
    """
    polygon : shapely.geometry
    edge_size : length of the grid cell
    """
    bounds = polygon.bounds
    x_rest = edge_size - (bounds[2]-bounds[0])%edge_size
    y_rest = edge_size - (bounds[3]-bounds[1])%edge_size
    x_coords = np.arange(bounds[0] - x_rest/2 + edge_size/2, bounds[2] + x_rest/2, edge_size)
    y_coords = np.arange(bounds[1] - y_rest/2 + edge_size/2, bounds[3] + y_rest/2, edge_size)
    combinations = np.array(list(product(x_coords, y_coords)))
    squares = gpd.points_from_xy(combinations[:, 0], combinations[:, 1]).buffer(edge_size / 2, cap_style=3)
    return gpd.GeoSeries(squares[squares.intersects(polygon)])

In [ ]:
utrecht_grid = make_grid_update(project_gemeenten.loc['Utrecht'].convexhull, .100)
random_gridvalues = gpd.GeoDataFrame(data= {'rand_value' : [22, 30, 12, 39, 49, 80]}, geometry = utrecht_grid)
random_gridvalues.crs = "EPSG:4326"

In [ ]:
Utrecht_edges = ox.utils_graph.graph_to_gdfs(Utrecht_network, nodes=False, edges=True, node_geometry=False)

In [ ]:
Utrecht_edges.to_file(r"data/utrecht_edges.geojson")

In [ ]:
Utrecht_edges.set_index(pd.Index([*range(1, len(Utrecht_edges)+1)]), append=True, inplace=True)
Utrecht_edges.head(50)

In [ ]:
Utrecht_edges.reset_index(inplace=True)
Utrecht_edges.head(50)

In [ ]:
Utrecht_edges[Utrecht_edges.level_3 == 6949]

In [ ]:
edges_w_randomvalues = gpd.overlay(Utrecht_edges, random_gridvalues,)

In [ ]:
edges_w_randomvalues.set_index("level_3", inplace=True)

In [ ]:
edges_w_randomvalues

In [ ]:
edges_w_randomvalues = gpd.sjoin(Utrecht_edges, random_gridvalues)

In [ ]:
edges_w_randomvalues.drop(columns=["oneway", "maxspeed", "lanes", "tunnel", "service", "width", "access", "junction", "est_width", "ref"], inplace=True)

In [ ]:
edges_w_randomvalues.drop(columns = "name", inplace=True)

In [ ]:
u_xml_all = ox.graph_from_xml("data/OSM/utrecht-latest.osm")

In [ ]:
edges_w_randomvalues.to_file(r"data/utrecht_edges.geojson")

In [ ]:
for i in edges_w_randomvalues.set_index(pd.Index([*range(1, len(edges_w_randomvalues)+1)]), append=True).index.duplicated(keep=False):
    if i == True:
        print(i)

In [ ]:
index = 0
for i in edges_w_randomvalues.index.duplicated(keep=False):
    if i == True:
        print(i)
        print(index)
    index += 1
    

In [ ]:
test_network = ox.graph_from_polygon(project_gemeenten.loc["Utrecht"].convexhull, network_type=("bike", "walk"))

### GRAPH FROM GEOPACKAGE

In [ ]:
gdf_nodes = gpd.read_file(fp, layer='nodes').set_index('osmid')
gdf_edges = gpd.read_file(fp, layer='edges').set_index(['u', 'v', 'key'])
assert gdf_nodes.index.is_unique and gdf_edges.index.is_unique

# convert the node/edge GeoDataFrames to a MultiDiGraph
graph_attrs = {'crs': 'epsg:4326', 'simplified': True}
G2 = ox.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs)